# Testing

We perform testing to increase confidence that our programs work correctly and that those programs meet our customer's expectations. We call this type of testing _functional testing_ as it ensures that the code functions correctly. While it sounds trite, the importance of software testing cannot be overstated.  Software and computer systems permeate our daily lives.  Unless you are reading these words from a dead tree off the grid somewhere, you are either working with a computer system or benefitting from a product (electricity) controlled by a computer system. Not only can software defects waste time as we work through the issues, those defects can lead monetary loss (both direct and indirect) as well as possibly deaths.  

Testing directly improves the product quality which in turn leads to higher customer satisfaction with systems.  Imagine using a computer that crashed on a regular basis.  How long would you continue to use that system.  Testing also decreases product development costs as finding and fixing defects sooner to the time when they are introduced into the system costs lest money to fix.

Yes, initially, you can get away with manually testing your software as your write code.  (And in some ways, this is necessary - build a little, test that those parts work, and build some more.  Rinse, Lather, Repeat.). However, eventually you will have significant maintainability issues. As you make changes, how do you ensure those changes did not break something?

Prior to the early 2000's, most software and system testing was a manual process.  Individuals wrote test scripts that were then executed.  Some unit test cases were built, but largely executed manually.  Since then, testing has been become automated (although not universally).  

Several advantages to automated testing:
1. Cost-savings.  These test cases can be run repetitively.  Yes, a higher upfront costs exists to develop test cases, but that is more of a one time expense.  Over the lifetime of a project, these test cases can now be executed thousands at time.
2. Faster development timeframes.  As enhancements are made to system, existing test cases can be executed to ensure the existing functionality still works.  (regression testing)
3. Immediate feedback
4. Automation of test cases development.  Fuzzing - https://owasp.org/www-community/Fuzzing
5. Automated testing is to foundation to continuous integration, continuous delivery, and other modern [DevOps practices](https://about.gitlab.com/topics/devops/).
6. Oh...  by the way, the automated grading of your program submissions - that's all unit tests.


This notebook will focus on verification of the code we have been developing - primarily this involves unit testing of the code we write. Generally speaking, unit tests check that the software produces the correct output based upon a specific input.  We compare actual and expected results to determine if an error occurred.  In terms of what exactly is a "unit", we consider this to be any portion of code that can be executed in isolation.  We can call functions and execute them, so functions are units. However, we cannot call specific lines within a function, so they are smaller than a unit.  As we get into classes, we will find that they have many pieces that can be tested in isolation. However, we can treat clases as units for testing.  

This notebook will walk through developing test cases that can be run on an automated basis. Our overarching goal is two-fold:
1. Deliver high-quality products to our customers
2. Reduce costs as much as possible.

We will use Python's built-in testing framework, `unittest`.  This framework provides capabilities to organize test cases, to execute code to setup any pre-conditions that may necessary to perform tests, and then code to remove any artifacts of the testing process.  The test cases work by making assertions about the code - generally does this result match some expected value. An assertion is a statement of fact<sup>[*](https://www.lexico.com/en/definition/assertion)</sup>. As such, if that assertion does not hold, either our assumption is incorrect or the code produced an incorrect value.

[unittest documentation](https://docs.python.org/3/library/unittest.html)

## Case Study: Bond Valuation
To develop some code to test, we will implement several functions to compute varies yield values for bonds.  

A bond represents a type of corporate debt.  A corporation issues a bond with a fixed principal that will be paid to the bond owner at maturity along with fixed interest payments along the way.  For instance, a corporation can issue a $\$$1,000 bond with 10$\%$ annual interest with a maturity of 10 years.  If an investor holds on to this bond, he or she wil receive $\$$2,000 over the lifetime (10 payments of $\$$100 plus $\$1,000 at the end).

One valuation to examine is the bond yield.  $bond yield = \frac{annual coupon payment}{bond price} $  where the $annual coupon payment = face value * coupon rate$

We can then define a function to implement the bond yield.

In [52]:
def compute_bond_yield(bond_price, face_value, coupon_rate):
    annual_coupon_payment = face_value * coupon_rate
    bond_yield = annual_coupon_payment / face_value         # used face_value instead of bond price
    return bond_yield

In [26]:
compute_bond_yield(970,1000,0.05)

0.05

The result looks correct, but is it really?  Ideally, we want to ensure that the output matches are expectation.  If we plug those numbers into a bond yield calculater,  we see that the result is actually 0.0515 

To help find issues like this, we need to write unit tests.  In fact, we can actually write these tests firsts and then build the function.  We know our functions then work when they pass the test cases.

To use the `unittest` module, we will first need to import that module and then write test cases. As with exceptions, we will need to extend a pre-existing class and then add methods to that class.  The approach will follow this outline:

<code>
    
import unittest

class Test<i>Name</i>(unittest.TestCase):
    def setUp(self):
        pass

    def tearDown(self):
        pass

    def test_<i>name_1</i>(self):
        ...
    
    def test_<i>name_2</i>(self):
        ...
    
    def test_<i>name_n</i>(self):
        ...
    
</code>  

Generally, you will want to clearly identify code that contains test cases. As such, it makes sense to start the class names for the unit test with "Test".  Similarly, we will also want to give descriptive names to the individual methods. Many testing frameworks assume methods starting with `test_` are test cases.  We can also add docstrings for the methods to provide a more detailed description into the output.

The `setup()` methods is called before each test method. Use this capability to allocate any resources or setup any conditions necessary to execute the test cases. You can also define a `setupClass()` method that runs once before any tests are executed in the class [more details](https://docs.python.org/3/library/unittest.html#unittest.TestCase.setUpClass). 

The `tearDown()` method is called after each test method, regardless of any exceptions.  Use method to release any allocated resources or move the system state back before the test cases executed. `tearDownClass()` also exists which executes after all of the tests in the class have executed.

It was not necessary to define `setup()` and `tearDown()` in these examples - the default behavior for both does nothing.


From within a Python program, we can execute the defined test cases with the following method call:
<code>
    
    unittest.main(argv=['unittest','Test<i>Name</i>'], verbosity=2, exit=False)

</code>

The 'TestName' can be removed to execute any test cases that have been loaded by the interpreter.

From the command-line, we can execute
`python -m unittest TestName`

To discover all possible tests cases and run those, use 
`python -m unittest discover`

So, now that we have seen the outline as to what needs to occur, let us write some actual test cases.

In [77]:
class TestBondYield(unittest.TestCase):
    "Validates compute_bond_yield"
    def setUp(self):
        pass
    def tearDown(self):
        pass
    
    def test_bond_yield_5_percent(self):
        "Validate that the computed bond yield approximates 0.0515"
        bond_yield = compute_bond_yield(970, 1000, 0.05)
        self.assertAlmostEqual(bond_yield, 0.0515, places=4)
        
    def test_bond_yield_0_percent(self):
        "Validate that the computed bond yield equals 0 for 0%"
        bond_yield = compute_bond_yield(970, 1000, 0.00)
        self.assertEqual(bond_yield, 0.0)        

In [78]:
unittest.main(argv=['unittest','TestBondYield'], verbosity=2, exit=False)

test_bond_yield_0_percent (__main__.TestBondYield)
Validate that the computed bond yield equals 0 for 0% ... ok
test_bond_yield_5_percent (__main__.TestBondYield)
Validate that the computed bond yield approximates 0.0515 ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


One of the tests passed, while the other test failed. We know need to debug the code to see what occured.  Since, this a relatively small code, we can employ the debug strategy of "read" and then "run".  As we read the source code, does it match up with the forumula?  No.  We can also set through it manually (either with a debugger or using a memory diagram / tracing approach).  

Let's correct the code and then re-run the test.

In [55]:
def compute_bond_yield(bond_price, face_value, coupon_rate):
    annual_coupon_payment = face_value * coupon_rate
    bond_yield = annual_coupon_payment / bond_price
    return bond_yield

In [56]:
unittest.main(argv=['unittest','TestBondYield'], verbosity=2, exit=False)

test_bond_yield_0_percent (__main__.TestBondYield)
Validate that the computed bond yield equals 0 for 0% ... ok
test_bond_yield_5_percent (__main__.TestBondYield)
Validate that the computed bond yield approximates 0.0515 ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


Another valuation for bonds is the yield to maturity(YTM).  This value is the speculative rate of a return given that an investor purchases a bond a given current market price and then holds the bond until marturity (thus, all interest payments and final payments are made).

The YTM can be approximated with this formula: 

$ YTM = \frac{C + \frac{FV-PV}{t}}{\frac{FV+PV}{2}} $
where 
- $C$ – Interest/coupon payment
- $FV$ – Face value of the security
- $PV$ – Present value/price of the security
- $t$ – How many years it takes the security to reach maturity

In [63]:
def calculate_yield_to_maturity(bond_price, face_value, coupon_rate, t):
    c = coupon_rate * face_value
    ytm = (c + (face_value - bond_price)/t) / ( (face_value+bond_price) /2 )
    return ytm

In [75]:
print(compute_bond_yield(850,1000,0.15))
print(calculate_yield_to_maturity(850,1000,0.15,7))

0.17647058823529413
0.18532818532818532


We can then develop another set of test cases to test that function:

In [99]:
class TestBondYTM(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    
    def test_bond_ytm_5_per_10_year(self):
        "Validate that the computed bond yield approximates 0.0515"
        bond_yield = calculate_yield_to_maturity(970, 1000, 0.05,10)
        self.assertAlmostEqual(bond_yield, 0.0538, places=4)
  

In [100]:
unittest.main(argv=['unittest','TestBondYTM'], verbosity=2, exit=False)

test_bond_ytm_5_per_10_year (__main__.TestBondYTM)
Validate that the computed bond yield approximates 0.0515 ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Documenting Test Cases
To document a test case, we generally define this information:
- Unique Identifier
- Test Inputs
- Expected Results
- Actual Results

As you look at the source coode, you can see that the unittest cases pretty much have this information in them already. You should include an identifier in the docstring - this will help you locate test cases in larger projects and possibly provide some sort of hierarchical orgnization if you so choose.  

If you do manually test code instead, you should keep track of your tests in a text file. That way, you at least have some reference as to how you tested the code before.  However, the *right* way to do this, is to create automated unit tests. The investment will be worth it.


## Black Box vs White Box Testing

test Equivalence Classes
- Input/output space is broken into different equivalence classes
- Each equivalence class is tested
- Tests are written to include “middle” input values from each of the possible classes – One test may consider multiple equivalence classes
  -– One for each type of input/output
  -- A test focuses on one equivalence class, but other values are needed for a full test. Those other values should be ”middle” values.
  -- Helps further test requirements by considering groups of inputs/outputs


Test Boundary Values
Programmers tend to make mistakes at boundaries
Want to test program boundaries and values to either side of the boundary

(smallest, largest, change from negative to positive)


white box

code coverage
- function/method
- statement
- edge
- branch
- condition. (predicate coverage)



## Testing Strategies

Testing Strategies
• Test Requirements
• Test Equivalence Classes • Test Boundary Values
• Test All Paths
• Test Exceptions

Reword all of the following and bring in more - testing strategy / developing test cases

1. Think about size. When a test involves a collection such as a list, string, dictionary, or file, you need to do the following:
  1. Test the empty collection.
  2. Test a collection with one item in it.
  3. Test a general case with several items.
  4. Test the smallest interesting case, such as sorting a list containing two values.

2. Think about dichotomies. A dichotomy is a contrast between two things. Examples of dichotomies are empty/full, even/odd, positive/negative, and alphabetic/nonalphabetic. If a function deals with two or more different categories or situations, make sure you test all of them.
3. Think about boundaries. If a function behaves differently around a particular boundary or threshold, test exactly that boundary case.
4. Think about order. If a function behaves differently when values appear in different orders, identify those orders and test each one of them. For the sorting example mentioned earlier, you’ll want one test case where the items are in order and one where they are not.

Paul Gries, Jennifer Campbell, and Jason Montojo. 2017. Practical Programming: An Introduction to Computer Science Using Python 3.6 (3rd. ed.). Pragmatic Bookshelf.

-----------
Whittaker’s book (Whittaker 2009) includes many examples of guidelines that can be used in test-case design. Some of the most general guidelines that he suggests are:

Choose inputs that force the system to generate all error messages:
Design inputs that cause input buffers to overflow.
Repeat the same input or series of inputs numerous times.
Force invalid outputs to be generated.
Force computation results to be too large or too small.

------



In [ ]:
How much to test?
Brooks’ Rule of Thumb for Scheduling a Software Project6
• 1/3 design
• 1/6 coding
• 1/2 testing
– 1/4 component testing (unit testing) – 1/4 system testing
Testing Early and Often
• Test early: start testing as soon as parts are implemented
• Test often: running tests at every reasonable opportunity
CSC116 Lecture Notes: Dr. Jessica Young Schmidt
  6F. P. Brooks, Jr., The Mythical Man-Month: Essays on Software Engineering, Anniversary Edition, Boston: Addison-Wesley, 1995.


`unittest` is not the only test framework available for Python.  We choose the framework as it is included in Python's standard library.  The framework also matches up similarly to the popular JUnit framework for Java.

Other frameworks to examine: 
- Pytest: https://docs.pytest.org/
- Hypothesis: https://hypothesis.readthedocs.io/

## Exercises


<pre>
